Localisation des dataset :

https://www.data.gouv.fr/datasets/demandes-de-valeurs-foncieres-geolocalisees/   
https://www.data.gouv.fr/datasets/population-municipale-des-communes-france-entiere/

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Affiche toutes les lignes
pd.set_option("display.max_rows", None)

# Affiche toutes les colonnes
pd.set_option("display.max_columns", None)

# Choisis le nombre de caractère par colonnes
pd.set_option("display.max_colwidth", 50)

In [ ]:
df_path = "full.csv"

df = pd.read_csv(df_path, low_memory=False)

df.columns

In [ ]:
col_to_keep = ["valeur_fonciere", "date_mutation", "id_mutation", "nature_mutation", "numero_disposition", "id_parcelle", "code_commune", "nom_commune", "code_postal", "type_local", "nature_culture", "nature_culture_speciale", 
               "nombre_lots", "nombre_pieces_principales", "surface_reelle_bati", "surface_terrain"]

df = df[col_to_keep]

In [ ]:
# garder uniquement les maisons et les appartements
type_local_to_keep = ['Maison', 'Appartement']
df = df[(df['type_local'].isin(type_local_to_keep))]

In [ ]:
# Vue d"ensemble (shape, dtypes)
def quick_overview(df, name):
    print(f"\n{name.upper()} SHAPE: {df.shape}")
    display(df.head())
    print(f"{name.upper()} Dtypes: \n{df.dtypes.value_counts()}")
    display(df.dtypes)
    display(df.describe(include="all").T)
    
quick_overview(df, "Dataset")

In [ ]:
df.isna().sum()

**Valeurs manquantes à supprimer :**   
On pourrait remplir les valeurs manquantes de "code_postal" mais :
- Une commune peut avoir plusieurs codes postaux (donc il faut la localiser via ces coordonnées gps ou son adresse postale)
- 72 lignes est négligeable pour 1 milions de lignes

Pareil pour "nombre_pieces_principales" et "surface_reelle_bati " (remplacer par une médiane par exemple)

**Valeurs manquantes à remplir :**   
Dans "nature_culture ", "nature_culture_speciale" et "surface_terrain" les NaN correspondent à des valeurs = 0

In [ ]:
# Lignes à supprimer
nan_to_drop = ["valeur_fonciere", "code_postal", "nombre_pieces_principales", "surface_reelle_bati"]

df = df.dropna(subset=nan_to_drop)

# Lignes à remplir
nan_to_fill = ["nature_culture", "nature_culture_speciale", "surface_terrain"]

df[nan_to_fill] = df[nan_to_fill].fillna(0)

In [ ]:
df.isna().sum()

## Regroupement des biens d'une même parcelle

In [ ]:
group_keys = ["date_mutation", "id_mutation", "id_parcelle"]

In [ ]:
df_types_parcelle = (
    df.groupby(group_keys)["type_local"]
      .nunique()
      .reset_index(name="nb_types")
)

df_mix_parcelle = df_types_parcelle[df_types_parcelle["nb_types"] > 1]

print(f"Nombre de parcelles avec plusieurs types de bien : {len(df_mix_parcelle)}")

In [ ]:
df_code_postale = (
    df.groupby(group_keys)["code_postal"]
      .nunique()
      .reset_index(name="nb_types")
)

df_mix_code_postale = df_code_postale[df_code_postale["nb_types"] > 1]

print(f"Nombre de parcelle avec plusieurs codes postaux : {len(df_mix_code_postale)}")
print(df_mix_code_postale.head())

In [ ]:
df[df["id_parcelle"] == "97416000CX0211"]

Saint-Pierre est une commune de la réunion avec les codes postaux 97410 et 97432.

La mutation avec plusieurs codes postaux diffèrents des autres mutations de la parcelle :
- 10 lignes pour cette mutation (1 ligne pour les autres)
- La valeur foncière est 10 fois plus élevé
- Elle se fait deux jours après une autre mutation

On peut supposer :
- Il y a eu une erreur dans le remplissage du bien
- La parcelle englobe plus qu'un bien qui serait à la limite des deux adresses postaux

On a quatres choix :
- Prendre 97432 car toutes les autres mutation de cette parcelle sont sous ce code postale
- Prendre 97410 car s'est le mode de cette mutation
- Séparer la mutation en deux et pondérer la valeur fonciere en fonction de la surface du bien
- Supprimer les lignes de cette mutation car modifier ou séparer la mutation pourrait altérer la prédiction

In [ ]:
# Suppression des lignes de la mutation
df = df[df["id_mutation"] != "2024-1179133"]

In [ ]:
df_code_commune = (
    df.groupby(group_keys)["code_commune"]
      .nunique()
      .reset_index(name="nb_types")
)

df_mix_code_commune = df_code_commune[df_code_commune["nb_types"] > 1]

print(f"Nombre de parcelle avec plusieurs codes de communes : {len(df_mix_code_commune)}")

**Description des colonnes**
- "date_mutation" : jour où le bien a été muté de propriétaire
- "id_mutation" : identification de la mutation
- "id_parcelle" : identification du cadastre du bien
- "numero_logement" : nombre de logement regroupe chaque mutations
- "nombre_lots" : nombre de copropriété attachés à une disposition

In [ ]:
# Crée une liste avec les valeurs str uniques
def join_type(x: pd.Series) -> str:
    vals = x.astype(str).unique()
    return ", ".join(sorted(vals))

df_agg = df.groupby(group_keys).agg({"numero_disposition": "count", "code_commune": "first", "surface_reelle_bati": "sum", 
                                     "surface_terrain": "sum", "nombre_pieces_principales": "sum",
                                     "type_local": join_type, "valeur_fonciere": "first"
                                    }).reset_index()

df_agg.rename(columns={"numero_disposition": "nombre_logement"}, inplace=True)

df_agg.head(10)

## Suppression des bien sans pièce principale

In [ ]:
(df_agg["nombre_pieces_principales"] == 0).sum()

In [ ]:
df_agg = df_agg[df_agg["nombre_pieces_principales"] != 0]

## Transformation du code commune en données analysable

Ce dataset regroupe toutes les communes de francais (outre mer inclue) et donne différentes informations utiles :
- Le numéro de région **(ancien regroupement à 27 région et non 18)**
- Le numéro de département
- La population en 2021   


**Point à prendre en compte :**
- La valeur foncière provient de mutation datant de 2024
- La population de chaque communes est une donnée de 2021
- Entre 2021 et 2024 la population française a eu une augmentation <=1%   
D'après ces données il peut y avoir altération sur la population des communes en 2024 (qu'on va décrire comme négligeable pour notre étude)

In [ ]:
df_path2 = "POPULATION_MUNICIPALE_COMMUNES_FRANCE.xlsx"

df2 = pd.read_excel(df_path2)

In [ ]:
df2.head()

In [ ]:
df_agg = df_agg.merge(
    df2[["codgeo", "reg", "dep", "p21_pop"]],
    left_on="code_commune",
    right_on="codgeo",
    how="inner"
)

df_agg = df_agg.drop(columns=["codgeo", "code_commune"])
df_agg = df_agg.rename(columns={"reg": "region", "dep": "departement", "p21_pop": "population_2021"})

print(df_agg.head())

Seuil de taille de ville :
- Petits villages : moins de 2 000 habitants
- Petites villes : 2 000 – 20 000 habitants
- Villes moyennes : 20 000 – 100 000 habitants
- Grandes villes : 100 000 – 500 000 habitants
- Très grandes villes : plus de 500 000 habitants

In [ ]:
def regroup_city(pop):
    if pop < 2000:
        return 1
    elif 2000 <= pop < 20000:
        return 2
    elif 20000 <= pop < 100000:
        return 3
    elif 100000 <= pop < 500000:
        return 4
    else:
        return 5
    
df_agg["cat_pop"] = df_agg["population_2021"].apply(regroup_city)
df_agg.drop(columns=["population_2021"])

## Ajout de données

In [ ]:
df_agg["room_size"] = df_agg["surface_reelle_bati"] / df_agg["nombre_pieces_principales"]
df_agg["room_by_housing"] = df_agg["nombre_pieces_principales"] / df_agg["nombre_logement"]
df_agg["squaremeter_built_by_housing"] = df_agg["surface_reelle_bati"] / df_agg["nombre_logement"]
df_agg["squaremeter_land_by_housing"] = df_agg["surface_terrain"] / df_agg["nombre_logement"]
df_agg["valeur_fonciere_by_housing"] = df_agg["valeur_fonciere"] / df_agg["nombre_logement"]

In [ ]:
to_del = ["surface_reelle_bati", "surface_terrain", "nombre_pieces_principales", "nombre_logement"]
df_agg = df_agg.drop(columns=to_del)

In [ ]:
df_agg["date_mutation"]= pd.to_datetime(df_agg["date_mutation"])

df_agg["annees_mutation"]= df_agg["date_mutation"].dt.year
df_agg["mois_mutation"]= df_agg["date_mutation"].dt.month
df_agg["jour_mutation"]= df_agg["date_mutation"].dt.day

df_agg.head()

## Analyse des features

In [ ]:
cat_data = ["type_local", "departement"]

num_data = df_agg.select_dtypes(include=["number"])
print(num_data.columns)

### Données qualitatives

In [ ]:
df_agg["type_local"].value_counts().plot(kind="bar")
plt.title(f"Répartition des mutation par type de parcelle")
plt.xlabel("Type de parcelle")
plt.ylabel("Fréquence")
plt.show()

In [ ]:
from matplotlib.ticker import MaxNLocator

ax = df_agg["departement"].value_counts().sort_index(ascending=True).plot(kind="bar")

plt.title("Répartition des mutations par département")
plt.xlabel("Numéro de département")
plt.ylabel("Fréquence")

# Forcer à n'afficher qu'environ 20% des ticks
ax.xaxis.set_major_locator(MaxNLocator(nbins=int(len(ax.get_xticks()) * 0.2)))

plt.xticks(rotation=45)
plt.show()

### Données quantitatives

In [ ]:
# for col in num_data.columns:
#     plt.figure(figsize=(8,4))
#     plt.hist(df_agg[col], bins=100) 
#     plt.title(f"Histogramme de {col}")
#     plt.xlabel(col)
#     plt.ylabel("Fréquence")
#     plt.show()

In [ ]:
import math

cols = num_data.columns
n = len(cols)

rows = math.ceil(n / 3)

fig, axes = plt.subplots(rows, 3, figsize=(18, 5*rows))
axes = axes.flatten()

for i, col in enumerate(cols):
    axes[i].hist(df_agg[col], bins=100)
    axes[i].set_title(f"Histogramme de {col}")
    axes[i].set_xlabel(col)
    axes[i].set_ylabel("Fréquence")

# cacher les axes vides s'il y en a
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

## Valeurs abérantes

D'après la loi française, un logement doit avoir au moins 9 m² habitables.       
De plus on peut supposer :
- Une valeur foncière d'une maison/appartement infèrieur à 1.000 est une erreur d'enregistrement.
- Un bien avec une surface > 1000m² ou une valeur foncière 2.000.000€ est un bien de luxe qu'on va considère comme valeur abérante

In [ ]:
((df_agg["squaremeter_built_by_housing"] < 9) | (df_agg["squaremeter_built_by_housing"] > 1000)).sum()

In [ ]:
((df_agg["valeur_fonciere_by_housing"] < 1000) | (df_agg["valeur_fonciere_by_housing"] > 2_000_000)).sum()

In [ ]:
df_agg = df_agg[(df_agg["squaremeter_built_by_housing"] >= 9) & (df_agg["squaremeter_built_by_housing"] <= 1000)]
df_agg = df_agg[(df_agg["valeur_fonciere_by_housing"] >= 1000) & (df_agg["valeur_fonciere_by_housing"] <= 2_000_000)]

In [ ]:
# Fonction pour enlever les outliers via l’IQR (Tukey's fences)
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]


colonnes = ["room_size", "room_by_housing", "squaremeter_land_by_housing"]
for col in colonnes:
    df_agg = remove_outliers_iqr(df_agg, col)

In [ ]:
cols = num_data.columns
n = len(cols)

rows = math.ceil(n / 3)

fig, axes = plt.subplots(rows, 3, figsize=(18, 5*rows))
axes = axes.flatten()

for i, col in enumerate(cols):
    axes[i].hist(df_agg[col], bins=100)
    axes[i].set_title(f"Histogramme de {col}")
    axes[i].set_xlabel(col)
    axes[i].set_ylabel("Fréquence")

# cacher les axes vides s'il y en a
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

## Encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder

to_encode = ["region", "type_local", "departement"]

encoder = OneHotEncoder(sparse_output=False)
encoded = encoder.fit_transform(df_agg[to_encode])

encoded_df = pd.DataFrame(
    encoded,
    columns=encoder.get_feature_names_out(to_encode),
    index=df_agg.index
)

df_encoded = pd.concat([df_agg.drop(columns=to_encode), encoded_df], axis=1)

df_encoded.head(10)

In [ ]:
# Suppression col ibutile ou qui peut "spoil" le target
col_to_drop = ["date_mutation", "id_mutation", "id_parcelle", "population_2021", "annees_mutation", "valeur_fonciere_by_housing"]
df_encoded = df_encoded.drop(columns=col_to_drop)

## Premier modele

In [ ]:
X = df_encoded.drop(columns=["valeur_fonciere"]).copy()
y = df_encoded["valeur_fonciere"].copy()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)